# IASI functions

In [ ]:
def IASI_version(component_nom, year):

    """ Get version of IASI dataset for each component nomenclature

         Args:
            component_nom (str): Component chemical nomenclature
            year (str): Year of dataset

        Returns:
            version (str): IASI dataset version
    """

    # https://iasi.aeris-data.fr/CO_IASI_C_L3_data/
    if component_nom == 'CO':
        version = 'V6.5.0'

    # https://iasi.aeris-data.fr/O3_IASI_C_L3_data/
    elif component_nom == 'O3':
        version = 'V6.5.1'

    # https://iasi.aeris-data.fr/NH3_IASI_C_L3_data/
    elif component_nom == 'NH3':
        version = 'V3.0.0'

    # https://iasi.aeris-data.fr/HCOOH_IASI_C_L3_data/
    elif component_nom == 'HCOOH':
        version = 'V1.0.0'

    return version

In [ ]:
def IASI_L3_download(component_nom, date, satellite):

    """ Download the IASI dataset with curl

         Args:
            component_nom (str): Component chemical nomenclature
            date (str): Query month as date

        Returns:
            product_name (str): Product name of IASI product
    """

    cnl = component_nom.lower()
    sl = 'iasi' + satellite.lower() + 'l3'

    year = date.split('-')[0]
    month = date.split('-')[1]
    version = IASI_version(component_nom, year)
    
    if component_nom == 'NH3':
        product_name = ''.join(['IASI_METOP' + satellite + '_L3_', component_nom, '_', year, month, '_ULB-LATMOS_', version, '.nc'])
    else:
        product_name = ''.join(['IASI_METOP' + satellite + '_L3_', component_nom, '_COLUMN_', year, month, '_ULB-LATMOS_', version, '.nc'])
    
    # Create directory for each satellite in case they do not exist
    path = os.path.join(os.path.abspath(''), 'data/' + sensor + '/' + component_nom + '/monthly/' + satellite)
    os.makedirs(path, exist_ok = True)

    !curl --insecure https://cds-espri.ipsl.fr/$sl/iasi_$cnl/$version/$year/$product_name --output data/iasi/$component_nom/monthly/$satellite/$product_name

In [ ]:
def IASI_L3_read(component_nom, satellites, dates):

    """ Read the IASI dataset as xarray dataset object and assign time

         Args:
            component_nom (str): Component chemical nomenclature
            satellites (list): List with A, B and/or C referring to METOP series
            dates (list): Query year-month

        Returns:
            sensor_ds (xarray): IASI dataset in xarray format
            sensor_type (str): Sensor type
    """

    sensor_ds_all = []

    for date in dates:

        year = date.split('-')[0]
        month = date.split('-')[1]

        sensor_ds_ABC = []
         
        for satellite in satellites:

            path = 'data/iasi/' + component_nom + '/monthly/' + satellite
            product_names = [file for file in os.listdir(path)]
            
            for product_name in product_names:

                sensor_ds_sat = xr.open_dataset('data/iasi/' + component_nom + '/monthly/' + satellite + '/' + product_name)
                sensor_ds_ABC.append(sensor_ds_sat)

        sensor_ds_ABC = xr.concat(sensor_ds_ABC, dim = 'latitude')
        time_str = dt.datetime(int(year), int(month), 1)
        sensor_ds_ABC = sensor_ds_ABC.assign_coords({'time': time_str}).expand_dims(dim = ['time'])
        sensor_ds_all.append(sensor_ds_ABC)
    
    sensor_ds = xr.concat(sensor_ds_all, dim = 'time')

    return sensor_ds